# Pcaps

In [1]:
import urllib.parse
import pprint

from scapy.all import *

In [2]:
# read a malicious packet capture downloaded from https://malware-traffic-analysis.net/
pcap = rdpcap(
    "../data/blog_show_data/2023-01-23-Google-ad-to-possible-TA505-activity.pcap"
)
type(pcap)

scapy.plist.PacketList

In [34]:
def parse_http_request(http_request: dict) -> dict:
    """_summary_

    Args:
        http_request (dict): _description_
        body (str): _description_

    Returns:
        dict: _description_
    """
    try:
        # this generates a ValueError if the http request does not have the proper structure
        method, path, protocol = http_request["http_fields"][0].split(" ")
        headers = {}
        for header in http_request[1:]:
            if header:
                header_name, header_value = header.split(":", maxsplit=1)
                headers[header_name] = header_value.strip()
        parsed_path = urllib.parse.urlparse(path)
        return {
            "timestamp": str(packet.time),
            "source_ip": packet[IP].src,
            "dest_ip": packet[IP].dst,
            "source_port": packet[TCP].sport,
            "dest_port": packet[TCP].dport,
            "method": method,
            "path": parsed_path.path,
            "query_string": parsed_path.query,
            "protocol": protocol,
            "headers": headers,
            "body": http_request.get("body"),
        }
    except:
        # http header may be non-standard, may have no method or may be  error code like 302
        return {
            "timestamp": str(packet.time),
            "source_ip": packet[IP].src,
            "dest_ip": packet[IP].dst,
            "source_port": packet[TCP].sport,
            "dest_port": packet[TCP].dport,
            "body": http_request.get("body"),
        }

In [36]:
def get_http(packet: scapy.layers.l2.Ether) -> dict:
    """_summary_

    Args:
        packet (scapy.layers.l2.Ether): _description_

    Returns:
        tuple: _description_
    """
    http_fields = []
    body = None

    # check if there is payload in packet
    if packet.haslayer(TCP) and packet.haslayer(Raw):
        try:
            # this generates UnicodeDecodeError if the Raw layer is not formatted properly
            payload = str(packet[Raw].load, "utf-8")

            if "HTTP/" in payload:
                if "\r\n\r\n" in payload:
                    body = payload.split("\r\n\r\n")[1]
                else:
                    body = payload.split("<!DOCTYPE html>")[1]

                # split each row of each http request, make a list of strings with these rows
                http_fields = payload.split("\r\n")
        except:
            return

    return {"http_fields": http_fields, "body": body}

In [37]:
for packet in pcap:
    http_request = get_http(packet)
    if http_request and http_request.get("http_fields"):
        parsed_payload = parse_http_request(http_request)
        if parsed_payload != None:
            pprint.pprint(parsed_payload)

{'body': '',
 'dest_ip': '191.101.13.129',
 'dest_port': 80,
 'source_ip': '10.1.23.101',
 'source_port': 49845,
 'timestamp': '1674496371.956557'}
{'body': '<!DOCTYPE html>\n'
         '<html style="height:100%">\n'
         '<head>\n'
         '<meta name="viewport" content="width=device-width, initial-scale=1, '
         'shrink-to-fit=no" />\n'
         '<title> 301 Moved Permanently\r\n'
         '</title></head>\n'
         '<body style="color: #444; margin:0;font: normal 14px/20px Arial, '
         'Helvetica, sans-serif; height:100%; background-color: #fff;">\n'
         '<div style="height:auto; min-height:100%; ">     <div '
         'style="text-align: center; width:800px; margin-left: -400px; '
         'position:absolute; top: 30%; left:50%;">\n'
         '        <h1 style="margin:0; font-size:150px; line-height:150px; '
         'font-weight:bold;">301</h1>\n'
         '<h2 style="margin-top:20px;font-size: 30px;">Moved Permanently\r\n'
         '</h2>\n'
         '<p>Th

# Netflow